In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import torch

import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from transformers import (BertForSequenceClassification, BertTokenizer,
                          RobertaForSequenceClassification, RobertaTokenizer,
                          XLNetForSequenceClassification, XLNetTokenizer,
                          AlbertForSequenceClassification, AlbertTokenizer,
                          AdamW, get_linear_schedule_with_warmup
                          )

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\n', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [lemmatizer.lemmatize(stemmer.stem(word)) for word in words if word not in stop_words]

    return ' '.join(words)

def encode_label(label:str):
    if label == 'true': return 0
    if label == 'mostly-true': return 1
    if label == 'barely-true': return 2
    if label == 'half-true': return 3
    if label == 'false': return 4
    if label == 'pants-fire': return 5
    return -1

def tokenize(X, y):
    input_ids = []
    attention_masks = []

    for txt in X.tolist():
        encoded_text = tokenizer.encode_plus(
                            txt,
                            add_special_tokens = True,
                            max_length = 100,
                            pad_to_max_length = True,
                            return_attention_mask = True,
                            return_tensors = 'pt'
                    )
        input_ids.append(encoded_text['input_ids'])
        attention_masks.append(encoded_text['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(torch.from_numpy(y.to_numpy()))
    
    return input_ids, attention_masks, labels

def accuracy(pred, actual):
    pred_flat = np.argmax(pred, axis=1).flatten()
    labels_flat = actual.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

### Load Data

In [17]:
df = pd.read_csv('../data/enriched_politifact.csv')
df = df.dropna(subset='speaker').reset_index().drop(columns='index')
df.head(3)

,source,context,target,speaker,documented_time,author_score,headline,article,summary,src_label,topic
0,Instagram posts,"stated on October 28, 2023 in a screenshot sha...",4,Madison Czopek,"October 31, 2023",[ 5 3 16 54 473 152],haaretz investig reveal discrep israel report ...,viral oct social medium post claim israel lie ...,haaretz isra newspap said x claim report blata...,4,81_gaza_palestinian_israelpalestin_israel
1,Scott Walker,"stated on May 30, 2023 in Interview:",2,Laura Schulte,"October 31, 2023",[26 45 39 41 44 11],wisconsin histor think larg continu blue state,wisconsin help swing presidenti vote donald tr...,although wisconsin vote democrat presidenti ca...,1,3_wisconsin_governor_walker_republican
2,Instagram posts,"stated on October 27, 2023 in a post:",4,Ciara O'Rourke,"October 30, 2023",[ 5 3 16 54 473 152],airport salzburg austria counter peopl flew au...,social medium post poi encourag peopl unfortun...,social medium post poi encourag peopl unfortun...,4,-1_barack_obama_clinton_democrat


In [14]:
def data_split(X, y, shuffle_idx, num_cols = 1):
    cols = ['headline', 'source', 'speaker', 'context', 'topic', 'summary']

    if(num_cols < 0 or num_cols > len(cols)):
        print("Invalid number of columns. [1-6]")
        return
    
    X_subset = ''
    for i in range(num_cols):
        X_subset += X[cols[i]] + ' '

    train_idx = shuffle_idx[:int(len(y) * 0.64)]
    valid_idx = shuffle_idx[int(len(y) * 0.64)+1 : int(len(y) * 0.8)]
    test_idx = shuffle_idx[int(len(y) * 0.8)+1:]

    return X_subset[train_idx], X_subset[valid_idx], X_subset[test_idx], y[train_idx], y[valid_idx], y[test_idx]

In [18]:
import random
X, y = df, df['target']
idx = list(range(df.shape[0]))
random.shuffle(idx)

In [ ]:
# Use TFIDF & Traditional ML

### XLNet Experiment

In [19]:
# Imbalanced Class Distribution

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn import metrics

# Loop through 6 columns
for i in range(1, 7):
    X_train, X_valid, X_test, y_train, y_valid, y_test = data_split(X, y, idx, num_cols=i)
    train_input_ids, train_attention_masks, train_labels = tokenize(X_train, y_train)
    valid_input_ids, valid_attention_masks, valid_labels = tokenize(X_valid, y_valid)
    test_input_ids, test_attention_masks, test_labels = tokenize(X_test, y_test)

    train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
    valid_dataset = TensorDataset(valid_input_ids, valid_attention_masks, valid_labels)
    test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

    epochs = 3
    batch_size = 8
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset),
                batch_size = batch_size)
    valid_dataloader = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size = batch_size)
    test_dataloader = DataLoader(
                test_dataset,
                sampler = SequentialSampler(test_dataset),
                batch_size = batch_size)
    
    # Model
    model = XLNetForSequenceClassification.from_pretrained(
    "xlnet-base-cased",
    num_labels = 6, 
    output_attentions = False,
    output_hidden_states = False)

    desc = model.cuda()
    optimizer = AdamW(model.parameters(), lr = 2e-4, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = len(train_dataloader) * epochs)
    

    # Train
    seed_val = 30
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    for epoch_i in range(0, epochs):
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):

            if step % 1000 == 0 and not step == 0:
                print('Batch {} of {}.'.format(step, len(train_dataloader)))

            batch = tuple(t.to(device) for t in batch)
            b_ids, b_mask, b_labels = batch

            model.zero_grad()        
            loss, logits = model(b_ids, 
                                token_type_ids=None, 
                                attention_mask=b_mask, 
                                labels=b_labels,
                                return_dict=False)
            
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader) 
                
        print("Avg. Train Loss: {0:.2f}".format(avg_train_loss))
        print("=====Validating====")

        model.eval()
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in valid_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_ids, b_mask, b_labels = batch
            
            with torch.no_grad():        
                (loss, logits, _) = model(b_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_mask,
                                    labels=b_labels,
                                    return_dict=False)
            total_eval_loss += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            total_eval_accuracy += accuracy(logits, label_ids)
            
        avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
        print("Avg. Val Acc: {0:.2f}".format(avg_val_accuracy))

        avg_val_loss = total_eval_loss / len(valid_dataloader)
        print("Val Loss: {0:.2f}".format(avg_val_loss))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy
            }
        )

    print("")
    print("Training complete!")
    print('======== Training Stat for i = {} is: {} ========'.format(i, training_stats))


    # Eval
    pred_probas , labels = [], []

    for (step, batch) in enumerate(test_dataloader):
    
        if step % 1000 == 0 and not step == 0:
            print('Batch {} of {}.'.format(step, len(test_dataloader)))
        
        batch = tuple(t.to(device) for t in batch)
        b_ids, b_mask, b_labels = batch
    
        with torch.no_grad():
            outputs = model(b_ids, token_type_ids=None, attention_mask=b_mask)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
    
        pred_probas.append(logits)
        labels.append(label_ids)

    preds = np.concatenate(pred_probas, axis=0)
    ytrue = np.concatenate(labels, axis=0)
    ypred = np.argmax(preds, axis=1)

    f1 = metrics.f1_score(ytrue, ypred, average='macro')
    acc = metrics.accuracy_score(ytrue, ypred)
    precision = metrics.precision_score(ytrue, ypred, average='macro')
    recall = metrics.recall_score(ytrue, ypred, average='macro')
    cmatrix = metrics.confusion_matrix(ytrue, ypred)
    print('F1 score: {:.3}, Accuracy: {:.3}, Precision: {:.3}, Recall: {:.3}, \n Confusion Matrix: \n {}'.format(f1, acc, precision, recall, cmatrix))
    

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.78
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 2 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.76
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 3 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.74
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.73

Training complete!
======== Training Stat for i = 1 is: [{'epoch': 1, 'Training Loss': 1.7847257475535074, 'Valid. Loss': 1.740508392167244, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 2, 'Training Loss': 1.7598350903193156, 'Valid. Loss': 1.741771245816115, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 3, 'Training Loss': 1.744027058728536, 'Valid. Loss': 1.7313743501838081, 'Valid. Accur.': 0.2990405117270789}] ========
F1 score: 0.0764, Accuracy: 0.297, Precision: 0.0496, Recall: 0.167, 
 Confusion Matrix: 
 [[   0    0    0    0  517    0]
 [   0    0    0    0  679    0]
 [   0    0    0    0  724    0]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.79
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 2 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.76
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 3 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.74
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.73

Training complete!
======== Training Stat for i = 2 is: [{'epoch': 1, 'Training Loss': 1.7852687650044758, 'Valid. Loss': 1.7404871861309386, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 2, 'Training Loss': 1.7599436747868855, 'Valid. Loss': 1.7421754242768928, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 3, 'Training Loss': 1.7449733110427856, 'Valid. Loss': 1.7320966494363, 'Valid. Accur.': 0.2990405117270789}] ========
F1 score: 0.0764, Accuracy: 0.297, Precision: 0.0496, Recall: 0.167, 
 Confusion Matrix: 
 [[   0    0    0    0  517    0]
 [   0    0    0    0  679    0]
 [   0    0    0    0  724    0]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

...

======== Epoch 1 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.79
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 2 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.76
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 3 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.75
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.73

Training complete!
======== Training Stat for i = 4 is: [{'epoch': 1, 'Training Loss': 1.7855671171188354, 'Valid. Loss': 1.7394638869808172, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 2, 'Training Loss': 1.761478601582845, 'Valid. Loss': 1.743528849280465, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 3, 'Training Loss': 1.7482638015111287, 'Valid. Loss': 1.7316504269520612, 'Valid. Accur.': 0.2990405117270789}] ========
F1 score: 0.0764, Accuracy: 0.297, Precision: 0.0496, Recall: 0.167, 
 Confusion Matrix: 
 [[   0    0    0    0  517    0]
 [   0    0    0    0  679    0]
 [   0    0    0    0  724    0

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.79
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 2 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.76
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 3 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.75
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.73

Training complete!
======== Training Stat for i = 5 is: [{'epoch': 1, 'Training Loss': 1.7854980374654135, 'Valid. Loss': 1.7409092078585107, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 2, 'Training Loss': 1.7604555293401083, 'Valid. Loss': 1.7426364124456704, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 3, 'Training Loss': 1.7458740366617838, 'Valid. Loss': 1.7314910685329803, 'Valid. Accur.': 0.2990405117270789}] ========
F1 score: 0.0764, Accuracy: 0.297, Precision: 0.0496, Recall: 0.167, 
 Confusion Matrix: 
 [[   0    0    0    0  517    0]
 [   0    0    0    0  679    0]
 [   0    0    0    0  724   

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.79
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 2 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.76
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.74
======== Epoch 3 / 3 ========
Batch 1000 of 1875.
Avg. Train Loss: 1.75
=====Validating====
Avg. Val Acc: 0.30
Val Loss: 1.73

Training complete!
======== Training Stat for i = 6 is: [{'epoch': 1, 'Training Loss': 1.785743014717102, 'Valid. Loss': 1.740984964218221, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 2, 'Training Loss': 1.7606256395975748, 'Valid. Loss': 1.7423761579781962, 'Valid. Accur.': 0.2990405117270789}, {'epoch': 3, 'Training Loss': 1.7464404373804727, 'Valid. Loss': 1.7314699583216262, 'Valid. Accur.': 0.2990405117270789}] ========
F1 score: 0.0764, Accuracy: 0.297, Precision: 0.0496, Recall: 0.167, 
 Confusion Matrix: 
 [[   0    0    0    0  517    0]
 [   0    0    0    0  679    0]
 [   0    0    0    0  724    0

In [16]:
# Balanced Class Distribution

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import numpy as np
from sklearn import metrics

# Loop through 6 columns
for i in range(1, 7):
    X_train, X_valid, X_test, y_train, y_valid, y_test = data_split(X, y, idx, num_cols=i)
    train_input_ids, train_attention_masks, train_labels = tokenize(X_train, y_train)
    valid_input_ids, valid_attention_masks, valid_labels = tokenize(X_valid, y_valid)
    test_input_ids, test_attention_masks, test_labels = tokenize(X_test, y_test)

    train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
    valid_dataset = TensorDataset(valid_input_ids, valid_attention_masks, valid_labels)
    test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)

    epochs = 3
    batch_size = 8
    train_dataloader = DataLoader(
                train_dataset,
                sampler = RandomSampler(train_dataset),
                batch_size = batch_size)
    valid_dataloader = DataLoader(
                valid_dataset,
                sampler = SequentialSampler(valid_dataset),
                batch_size = batch_size)
    test_dataloader = DataLoader(
                test_dataset,
                sampler = SequentialSampler(test_dataset),
                batch_size = batch_size)
    
    # Model
    model = XLNetForSequenceClassification.from_pretrained(
    "xlnet-base-cased",
    num_labels = 6, 
    output_attentions = False,
    output_hidden_states = False)

    desc = model.cuda()
    optimizer = AdamW(model.parameters(), lr = 2e-4, eps = 1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0,
                                                num_training_steps = len(train_dataloader) * epochs)
    

    # Train
    seed_val = 30
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    for epoch_i in range(0, epochs):
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        total_train_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):

            if step % 1000 == 0 and not step == 0:
                print('Batch {} of {}.'.format(step, len(train_dataloader)))

            batch = tuple(t.to(device) for t in batch)
            b_ids, b_mask, b_labels = batch

            model.zero_grad()        
            loss, logits = model(b_ids, 
                                token_type_ids=None, 
                                attention_mask=b_mask, 
                                labels=b_labels,
                                return_dict=False)
            
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader) 
                
        print("Avg. Train Loss: {0:.2f}".format(avg_train_loss))
        print("=====Validating====")

        model.eval()
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        for batch in valid_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_ids, b_mask, b_labels = batch
            
            with torch.no_grad():        
                (loss, logits, _) = model(b_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_mask,
                                    labels=b_labels,
                                    return_dict=False)
            total_eval_loss += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            total_eval_accuracy += accuracy(logits, label_ids)
            
        avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
        print("Avg. Val Acc: {0:.2f}".format(avg_val_accuracy))

        avg_val_loss = total_eval_loss / len(valid_dataloader)
        print("Val Loss: {0:.2f}".format(avg_val_loss))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy
            }
        )

    print("")
    print("Training complete!")
    print('======== Training Stat for i = {} is: {} ========'.format(i, training_stats))


    # Eval
    pred_probas , labels = [], []

    for (step, batch) in enumerate(test_dataloader):
    
        if step % 1000 == 0 and not step == 0:
            print('Batch {} of {}.'.format(step, len(test_dataloader)))
        
        batch = tuple(t.to(device) for t in batch)
        b_ids, b_mask, b_labels = batch
    
        with torch.no_grad():
            outputs = model(b_ids, token_type_ids=None, attention_mask=b_mask)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
    
        pred_probas.append(logits)
        labels.append(label_ids)

    preds = np.concatenate(pred_probas, axis=0)
    ytrue = np.concatenate(labels, axis=0)
    ypred = np.argmax(preds, axis=1)

    f1 = metrics.f1_score(ytrue, ypred, average='macro')
    acc = metrics.accuracy_score(ytrue, ypred)
    precision = metrics.precision_score(ytrue, ypred, average='macro')
    recall = metrics.recall_score(ytrue, ypred, average='macro')
    cmatrix = metrics.confusion_matrix(ytrue, ypred)
    print('F1 score: {:.3}, Accuracy: {:.3}, Precision: {:.3}, Recall: {:.3}, \n Confusion Matrix: \n {}'.format(f1, acc, precision, recall, cmatrix))
    

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenc

======== Epoch 1 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.84
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.81
======== Epoch 2 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.81
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.79
======== Epoch 3 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.80
=====Validating====
Avg. Val Acc: 0.15
Val Loss: 1.79

Training complete!
======== Training Stat for i = 1 is: [{'epoch': 1, 'Training Loss': 1.844217666277445, 'Valid. Loss': 1.8073982676634421, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 2, 'Training Loss': 1.814271642788347, 'Valid. Loss': 1.7920895516872406, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 3, 'Training Loss': 1.8018984930582314, 'Valid. Loss': 1.7942728732640927, 'Valid. Accur.': 0.14623397435897437}] ========
F1 score: 0.0473, Accuracy: 0.165, Precision: 0.0276, Recall: 0.167, 
 Confusion Matrix: 
 [[  0 534   0   0   0   0]
 [  0 515   0   0   0   0]
 [  0 508   0   0   0   0]
 [  0 485   0

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.84
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.81
======== Epoch 2 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.82
=====Validating====
Avg. Val Acc: 0.16
Val Loss: 1.79
======== Epoch 3 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.80
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.79

Training complete!
======== Training Stat for i = 2 is: [{'epoch': 1, 'Training Loss': 1.8431228709508138, 'Valid. Loss': 1.8056501623911736, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 2, 'Training Loss': 1.8166435224464141, 'Valid. Loss': 1.791951359846653, 'Valid. Accur.': 0.16466346153846154}, {'epoch': 3, 'Training Loss': 1.8038822718892231, 'Valid. Loss': 1.794546332114782, 'Valid. Accur.': 0.17708333333333334}] ========
F1 score: 0.0449, Accuracy: 0.156, Precision: 0.026, Recall: 0.167, 
 Confusion Matrix: 
 [[  0   0   0 534   0   0]
 [  0   0   0 515   0   0]
 [  0   0   0 508   0   0]
 [  0   0   0 

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

...

======== Epoch 1 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.84
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.81
======== Epoch 2 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.82
=====Validating====
Avg. Val Acc: 0.16
Val Loss: 1.79
======== Epoch 3 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.81
=====Validating====
Avg. Val Acc: 0.17
Val Loss: 1.79

Training complete!
======== Training Stat for i = 4 is: [{'epoch': 1, 'Training Loss': 1.8441005306550298, 'Valid. Loss': 1.8059642586188438, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 2, 'Training Loss': 1.8187992663747337, 'Valid. Loss': 1.7912747791180244, 'Valid. Accur.': 0.16466346153846154}, {'epoch': 3, 'Training Loss': 1.8068362263790574, 'Valid. Loss': 1.7948151662563667, 'Valid. Accur.': 0.1658653846153846}] ========
F1 score: 0.0468, Accuracy: 0.163, Precision: 0.0272, Recall: 0.167, 
 Confusion Matrix: 
 [[  0   0 534   0   0   0]
 [  0   0 515   0   0   0]
 [  0   0 508   0   0   0]
 [  0   0 48

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.85
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.81
======== Epoch 2 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.82
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.79
======== Epoch 3 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.81
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.79

Training complete!
======== Training Stat for i = 5 is: [{'epoch': 1, 'Training Loss': 1.8452703753628406, 'Valid. Loss': 1.8070262101216195, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 2, 'Training Loss': 1.8193726408433724, 'Valid. Loss': 1.7908107390006383, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 3, 'Training Loss': 1.806865449794325, 'Valid. Loss': 1.794233045134789, 'Valid. Accur.': 0.17708333333333334}] ========
F1 score: 0.0449, Accuracy: 0.156, Precision: 0.026, Recall: 0.167, 
 Confusion Matrix: 
 [[  0   0   0 534   0   0]
 [  0   0   0 515   0   0]
 [  0   0   0 508   0   0]
 [  0   0   0 

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

======== Epoch 1 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.84
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.81
======== Epoch 2 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.82
=====Validating====
Avg. Val Acc: 0.18
Val Loss: 1.79
======== Epoch 3 / 3 ========
Batch 1000 of 1245.
Avg. Train Loss: 1.81
=====Validating====
Avg. Val Acc: 0.15
Val Loss: 1.79

Training complete!
======== Training Stat for i = 6 is: [{'epoch': 1, 'Training Loss': 1.8437713273557794, 'Valid. Loss': 1.8053208203651967, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 2, 'Training Loss': 1.8199051489312965, 'Valid. Loss': 1.7909569159532204, 'Valid. Accur.': 0.17708333333333334}, {'epoch': 3, 'Training Loss': 1.8077350846256117, 'Valid. Loss': 1.7945181352969928, 'Valid. Accur.': 0.14623397435897437}] ========
F1 score: 0.0473, Accuracy: 0.165, Precision: 0.0276, Recall: 0.167, 
 Confusion Matrix: 
 [[  0 534   0   0   0   0]
 [  0 515   0   0   0   0]
 [  0 508   0   0   0   0]
 [  0 485  

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
valid_dataset = TensorDataset(valid_input_ids, valid_attention_masks, valid_labels)

epochs = 3
batch_size = 8
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size)
valid_dataloader = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size = batch_size)


### Baseline BERT Model

In [ ]:
model = XLNetForSequenceClassification.from_pretrained(
    "xlnet-base-cased",
    num_labels = 6, 
    output_attentions = False,
    output_hidden_states = False)

desc = model.cuda()
optimizer = AdamW(model.parameters(), lr = 2e-4, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = len(train_dataloader) * epochs)

### Training

In [ ]:
import random
import numpy as np

seed_val = 30
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 500 == 0 and not step == 0:
            print('Batch {} of {}.'.format(step, len(train_dataloader)))

        batch = tuple(t.to(device) for t in batch)
        b_ids, b_mask, b_labels = batch

        model.zero_grad()        
        loss, logits = model(b_ids, 
                             token_type_ids=None, 
                             attention_mask=b_mask, 
                             labels=b_labels,
                             return_dict=False)
        
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader) 
               
    print("Avg. Train Loss: {0:.2f}".format(avg_train_loss))
    print("=====Validating====")

    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_ids, b_mask, b_labels = batch
        
        with torch.no_grad():        
            (loss, logits, _) = model(b_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_mask,
                                   labels=b_labels,
                                   return_dict=False)
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(valid_dataloader)
    print("Avg. Val Acc: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(valid_dataloader)
    print("Val Loss: {0:.2f}".format(avg_val_loss))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy
        }
    )

print("")
print("Training complete!")

### Evaluation

In [ ]:
pred_probas , labels = [], []

for (step, batch) in enumerate(valid_dataloader):
  
    if step % 500 == 0 and not step == 0:
        print('Batch {} of {}.'.format(step, len(valid_dataloader)))
    
    batch = tuple(t.to(device) for t in batch)
    b_ids, b_mask, b_labels = batch
  
    with torch.no_grad():
        outputs = model(b_ids, token_type_ids=None, attention_mask=b_mask)

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
  
    pred_probas.append(logits)
    labels.append(label_ids)

In [ ]:
preds = np.concatenate(pred_probas, axis=0)
ytrue = np.concatenate(labels, axis=0)
ypred = np.argmax(preds, axis=1)

In [ ]:
from sklearn import metrics

f1 = metrics.f1_score(ytrue, ypred, average='macro')
acc = metrics.accuracy_score(ytrue, ypred)
precision = metrics.precision_score(ytrue, ypred, average='macro')
recall = metrics.recall_score(ytrue, ypred, average='macro')
cmatrix = metrics.confusion_matrix(ytrue, ypred)
print('F1 score: {:.3}, Accuracy: {:.3}, Precision: {:.3}, Recall: {:.3}, \n Confusion Matrix: \n {}'.format(f1, acc, precision, recall, cmatrix))

### Factor Models

In [ ]:
from clickbait import ClickbaitModel
from sentiment_log import SentimentModel
from spam import SpamModel
from source_reliable import SourceReliableModel

X_train, X_test, y_train, y_test = train_test_split(df['headline'], df['target'], test_size=.2, random_state=11)

clickM = ClickbaitModel()
sentiM = SentimentModel()
spamM = SpamModel()
srcM = SourceReliableModel()

sentiM.fit(X_train)
spamM.fit(X_train)
srcM.fit(X_train)


In [ ]:
acc_lst = [0.946, 0.976, 0.412, 0.589]
weight = [acc/sum(acc_lst) for acc in acc_lst]
clickbaitV = clickM.predict(X_test)[1] * weight[0]
sentiV = sentiM.predict(X_test)[1] * weight[1]
spamV = spamM.predict(X_test)[1] * weight[2]
sourceV = srcM.predict(X_test)[1] * weight[3]

In [ ]:
train_clickbaitV = clickM.predict(X_train)[1] * weight[0]
train_sentiV = sentiM.predict(X_train)[1] * weight[1]
train_spamV = spamM.predict(X_train)[1] * weight[2]
train_sourceV = srcM.predict(X_train)[1] * weight[3]

In [ ]:
veracity_train = pd.DataFrame({'clickbait':train_clickbaitV, 'sentiment': train_sentiV, 'spam': train_spamV, 'source': train_sourceV})
veracity_test = pd.DataFrame({'clickbait':clickbaitV, 'sentiment': sentiV, 'spam': spamV, 'source': sourceV})

In [ ]:
clf = LogisticRegression(solver='liblinear')
clf.fit(veracity_train, y_train)
clf.score(veracity_test, y_test)